In [1]:
from sklearn import datasets
from sklearn.svm import SVC
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble  import RandomForestClassifier
from mipego4ml.ConditionalSpace import ConditionalSpace
from mipego4ml.ConfigSpace import ConfigSpace
from mipego4ml.mipego.SearchSpace import ContinuousSpace, NominalSpace, OrdinalSpace
from mipego4ml.mipego4ML import mipego4ML
import warnings
warnings.filterwarnings("ignore")

# Define Configuration Space

In [3]:
#define Configuration space
search_space = ConfigSpace()

#Define Search Space
alg_namestr = NominalSpace(["SVM", "RF"], "alg_namestr")

#Define Search Space for Support Vector Machine
kernel = NominalSpace(["linear", "rbf", "poly", "sigmoid"], "kernel")
C = ContinuousSpace([1e-2, 100], "C")
degree = OrdinalSpace([1, 5], 'degree')
coef0 = ContinuousSpace([0.0, 10.0], 'coef0') 
gamma = ContinuousSpace([0, 20], 'gamma')
# Define Search Space for Random Forest
n_estimators = OrdinalSpace([5, 100], "n_estimators")
criterion = NominalSpace(["gini", "entropy"], "criterion")
max_depth = OrdinalSpace([10, 200], "max_depth")
max_features = NominalSpace(['auto', 'sqrt', 'log2'], "max_features")

# Add Search space to Configuraion Space
search_space.add_multiparameter([alg_namestr, kernel, C, degree, coef0, gamma
                                        , n_estimators, criterion, max_depth, max_features])
#Define conditional Space
con = ConditionalSpace("conditional")
con.addMutilConditional([kernel, C, degree, coef0, gamma,], alg_namestr, "SVM")
con.addMutilConditional([n_estimators,criterion,max_depth,max_features],alg_namestr,"RF")

# Objective Function

In [4]:
iris = datasets.load_iris()
X = iris.data
y = iris.target

def obj_func(params):
    params = {k : params[k] for k in params if params[k]}
    #print(params)
    classifier = params['alg_namestr']
    params.pop("alg_namestr", None)    
    #print(params)
    clf = SVC()
    if(classifier == 'SVM'):
        clf = SVC(**params)
    elif(classifier == 'RF'):
        clf = RandomForestClassifier(**params)
    mean= cross_val_score(clf, X, y).mean()
    loss = 1 - mean
    #print (mean)
    return loss


# Run Optimise 

In [7]:
opt = mipego4ML(search_space, con, obj_func, minimize=True,  # the problem is a minimization problem.
                max_eval=100,  # we evaluate maximum 100 times
                max_iter=100,  # we have max 100 iterations
                infill='EI',  # Expected improvement as criteria
                n_init_sample=10,  # We start with 10 initial samples
                n_point=1,  # We evaluate every iteration 1 time
                n_job=1,  # with 1 process (job).
                optimizer='MIES',  # We use the MIES internal optimizer.
                verbose=False, random_seed=None)

incumbent, best_value = opt.run()
print("Best Value:", best_value)

The  1  round, Runing: 2 Candidates
previous best loss was: 0.02614379084967322 of 0
previous best loss was: 0.02614379084967322 of 1
The  2  round, Runing: 1 Candidates
previous best loss was: 0.02614379084967322 of 0
Best Value: 0.020016339869281086
